In [1]:
from src.annotation_handling.segmentsai_handler import SegmentsAIHandler
from src.pipelines import model_assist_label_pipeline as pipe 
import config

/home/etaylor/.conda/envs/detectron/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Setup Global Variables

In [2]:
segmentsai_handler = SegmentsAIHandler()
TRAIN_DATASET_NAME = 'etaylor/cannabis_patches_all_images'

# New dataset to upload variables
image_name = "IMG_2263"
week = 'week9'
zoom_type = '3xr'

#### 1. Upload your images and label a small subset

##### Vizualize dataset images with masks

In [ ]:
# Visualize the dataset
segmentsai_handler.visualize_dataset(TRAIN_DATASET_NAME)

#### 2. Train a segmentation model on the labeled images


In [4]:
# Initialize a dataset from the release file
model = pipe.train_segmentation_model(TRAIN_DATASET_NAME)

Initializing dataset...
Preloading all samples. This may take a while...


100%|██████████| 95/95 [00:05<00:00, 18.82it/s]


Initialized dataset with 95 images.
Exporting dataset. This may take a while...


100%|██████████| 95/95 [00:03<00:00, 29.85it/s]


Exported to ./export_coco-instance_etaylor_cannabis_patches_all_images_v0.2.json. Images in segments/etaylor_cannabis_patches_all_images/v0.2
Metadata(name='my_dataset', json_file='./export_coco-instance_etaylor_cannabis_patches_all_images_v0.2.json', image_root='segments/etaylor_cannabis_patches_all_images/v0.2', evaluator_type='coco', thing_classes=['trichome'])
[11/13 16:45:38 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
  

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[11/13 16:45:42 d2.engine.train_loop]: Starting training from iteration 0


/home/etaylor/.conda/envs/detectron/lib/python3.9/site-packages/detectron2/data/detection_utils.py:446: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  torch.stack([torch.from_numpy(np.ascontiguousarray(x)) for x in masks])
/home/etaylor/.conda/envs/detectron/lib/python3.9/site-packages/detectron2/data/detection_utils.py:446: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or mak

[11/13 16:45:48 d2.utils.events]:  eta: 0:00:50  iter: 19  total_loss: 3.877  loss_cls: 0.5899  loss_box_reg: 0.6108  loss_mask: 0.6931  loss_rpn_cls: 1.893  loss_rpn_loc: 0.1614    time: 0.1846  last_time: 0.2056  data_time: 0.0328  last_data_time: 0.0076   lr: 1.6068e-05  max_mem: 2009M
[11/13 16:45:56 d2.utils.events]:  eta: 0:00:47  iter: 39  total_loss: 2.234  loss_cls: 0.5555  loss_box_reg: 0.6792  loss_mask: 0.6616  loss_rpn_cls: 0.2181  loss_rpn_loc: 0.09586    time: 0.1902  last_time: 0.1601  data_time: 0.0056  last_data_time: 0.0043   lr: 3.2718e-05  max_mem: 2014M
[11/13 16:46:00 d2.utils.events]:  eta: 0:00:46  iter: 59  total_loss: 1.994  loss_cls: 0.5122  loss_box_reg: 0.7078  loss_mask: 0.5967  loss_rpn_cls: 0.1003  loss_rpn_loc: 0.09245    time: 0.1932  last_time: 0.2004  data_time: 0.0102  last_data_time: 0.0064   lr: 4.9367e-05  max_mem: 2014M
[11/13 16:46:04 d2.utils.events]:  eta: 0:00:42  iter: 79  total_loss: 1.872  loss_cls: 0.4724  loss_box_reg: 0.726  loss_mask

#### Create a new dataset and upload images to him

In [3]:
# Create a new test dataset for the specified image, week, and zoom type
test_dataset = pipe.create_new_test_dataset(image_name, config.WEEKS_DIR[week], config.ZOOM_TYPES_DIR[zoom_type])
# Get the absolute path to the processed image
abs_images_path = f"{config.get_processed_cannabis_image_path(week, zoom_type)}/{image_name}"

# Upload the images that are not annotated to the dataset
segmentsai_handler.upload_images(test_dataset, abs_images_path)

name='cannabis_patches_week9_15_06_2023_3x_regular_IMG_2263_v2' full_name='etaylor/cannabis_patches_week9_15_06_2023_3x_regular_IMG_2263_v2' cloned_from=None description='cannabis patches week=week9_15_06_2023 zoom_type=3x_regular of image=IMG_2263_v2.' category='other' public=False owner=Owner(username='etaylor', created_at='2022-12-28T12:53:18Z', email=None) created_at='2023-11-13T15:02:52.964595Z' enable_ratings=False enable_skip_labeling=True enable_skip_reviewing=False enable_save_button=False enable_label_status_verified=False enable_same_dimensions_track_constraint=False enable_interpolation=True task_type='segmentation-bitmap' label_stats=LabelStats(REVIEWED=None, REVIEWING_IN_PROGRESS=None, LABELED=None, LABELING_IN_PROGRESS=None, REJECTED=None, PRELABELED=None, SKIPPED=None, VERIFIED=None, UNLABELED=None, TOTAL=None) labeling_inactivity_timeout_seconds=None samples_count=0 collaborators_count=None task_attributes=TaskAttributes(format_version='0.1', categories=[TaskAttributeC

In [6]:
release_name = "v0.1"
description = "upload predictions to dataset."
segmentsai_handler.client.add_release(test_dataset, release_name, description)
test_release = segmentsai_handler.client.get_release(test_dataset, "v0.1")

In [9]:
test_release

Release(uuid='b79c169b-8ad3-41ad-99a9-eb3562c2e4d6', name='v0.1', description='upload predictions to dataset.', release_type='JSON', attributes=URL(url=''), status='PENDING', created_at='2023-11-13T14:51:44.611766Z', samples_count=13)

### Upload the Images that are not annotaed to the dataset

In [7]:
pipe.upload_predictions(test_release, model)

release=uuid='b79c169b-8ad3-41ad-99a9-eb3562c2e4d6' name='v0.1' description='upload predictions to dataset.' release_type='JSON' attributes=URL(url='') status='PENDING' created_at='2023-11-13T14:51:44.611766Z' samples_count=13


MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant https://?

In [ ]:
# Delete the dataset - ONLY IF YOU WANT TO DELETE THE DATASET!!!
DELETE_DATASET_NAME = ""
segmentsai_handler.client.delete_dataset(DELETE_DATASET_NAME)